In [4]:
pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 20.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 20.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [49]:
import os
import time
import copy
import random
import math
import os
import time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import IPython

In [50]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Start Connecting

In [16]:
train_dataset_CIFAR10 = datasets.CIFAR10(root='data',
                                       train=True,
                                       transform=transforms.ToTensor(),
                                       download=True)
test_dataset_CIFAR10 = datasets.CIFAR10(root='data',
                                      train=False,
                                      transform=transforms.ToTensor(),
                                      download=True)
train_loader_CIFAR10 = DataLoader(dataset=train_dataset_CIFAR10,
                                  batch_size=64,
                                  shuffle=True)
test_loader_CIFAR10 = DataLoader(dataset=test_dataset_CIFAR10,
                                 batch_size=64,
                                 shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [69]:
data_tensor, label_tensor = torch.load('/workspace/Taein/jooyeong/data/cifar10_split_1.pt')

# 텐서 데이터를 데이터셋으로 변환
client_dataset = TensorDataset(torch.stack(data_tensor), torch.tensor(label_tensor))

# DataLoader를 사용하여 데이터셋 로드
client_loader = DataLoader(client_dataset, batch_size=32, shuffle=True)

/tmp/ipykernel_1036/2134898122.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_tensor, label_tensor = torch.load('/workspace/Taein/jooyeong/data/cifar10_split_1.pt'

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

# ResNet18 Model

In [37]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock_18(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock_18, self).__init__()
        self.conv_1 = conv3x3(inplanes, planes, stride)
        self.bn_1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv_2 = conv3x3(planes, planes)
        self.bn_2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv_1(x)
        out = self.bn_1(out)
        out = self.relu(out)

        out = self.conv_2(out)
        out = self.bn_2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        out = self.relu(out)

        return out

class ResNet_18(nn.Module):
    def __init__(self, block, layers, num_classes, grayscale):
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3

        super(ResNet_18, self).__init__()
        self.conv1g = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bng = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512*block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes*block.expansion),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1g(x)
        x = self.bng(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x_full = x.view(x.size(0), -1)
        logits = self.fc(x_full)
        #probas = F.softmax(logits, dim=1)
        return logits, x_full

def resnet18(num_classes, grayscale):
    model = ResNet_18(block=BasicBlock_18, layers=[2,2,2,2], num_classes=num_classes, grayscale=grayscale)
    return model

# Client Model Training

In [9]:
def avg_train_client(id, client_loader, global_model, num_local_epochs, lr):
    local_model = copy.deepcopy(global_model)
    local_model = local_model.to(device)
    local_model.train()
    optimizer = torch.optim.SGD(local_model.parameters(), lr=lr, momentum = 0.9)

    for epoch in range(num_local_epochs):
        print('    Epoch {}'.format(epoch+1))
        for (i, (x,y)) in enumerate(client_loader):
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            local_out, _ = local_model(x)
      
            loss = criterion(local_out, y)
            loss.backward()
            optimizer.step()
            

    
    
    return local_model

In [19]:


### 서버에게서 받은 global model.pt
round = 0
while True:
    
    data = s.recv(1024)
    if int(data.decode('utf-8')) != round:
        print("라운드 동기 오류")
        exit(1)
    
    client_model = torch.load('/model/global_model.pt')
    #client_model = resnet18(10, False)
    #client_model.load_state_dict(global_model)

    
    client_dataset = torch.load('/data/cifar10_split_1.pt')
    client_loader = DataLoader(client_dataset, batch_size=32, shuffle=True)


    criterion = nn.CrossEntropyLoss()

    client_model_1 = avg_train_client(1, client_loader, client_model, 5, 0.01)
    
    file_path = f'/model/client_model_{client_num}.pt'
    torch.save(client_model_1, file_path)

    s.sendall(data.decode('utf-8'))
    round += 1
### 클라이언트는 서버에게 client_model_(i).pt를 준다.

OSError: [Errno 9] Bad file descriptor

In [ ]:
# 1 라운드 동안.. 접속 완료
# 서버에게 모델을 받음 
# 함수 돌리고
# 모델 저장
# 서버에게 보냄

In [18]:
import socket


server_address = ('10.3.128.1', 9090)

print("Trying to connect...")

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    try:
        # 서버에 연결 시도 
        s.connect(server_address)
        
        # 연결 성공 시 서버 정보 출력
        
        print(f"연결이 완료되었습니다. 서버 주소: {server_address[0]}, 포트: {server_address[1]}")
        print(f"로컬 주소: {s.getsockname()}")  # 클라이언트(로컬) 주소 출력
        
        # 서버로부터 숫자 수신
        received_data = s.recv(1024)  # 서버로부터 최대 1024바이트를 수신
        received_str = received_data.decode('utf-8')  # 받은 데이터를 문자열로 변환
        
        # 문자열 데이터를 정수로 변환
        client_num = int(received_str)  # 문자열을 정수로 변환
        print(f"서버로부터 받은 숫자(정수): {client_num}")  # 정수로 출력
            
    except socket.error as e:
        # 연결 실패 시 오류 메시지 출력
        print(f"서버에 연결할 수 없습니다: {e}")


Trying to connect...
서버에 연결할 수 없습니다: [Errno 111] Connection refused


In [40]:
import socket

server_address = ('10.3.129.180', 9090)

try:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect(server_address)
    print('서버와 연결되었습니다.')

    while True:
        try:
            data = s.recv(1024)
            print('test1', data)
            if not data:
                print('서버로부터 데이터를 수신할 수 없습니다.')
                break

            receive_round = int(data.decode('utf-8'))

            s.sendall(str(1).encode())
            print("정보를 서버에게 보냈습니다.")
        except socket.error as e:
            print(f"소켓 오류 발생: {e}")

        try:
            data = s.recv(1024)
            data = int(data.decode('utf-8'))
            if data == 1:
                print("클라이언트 학습 시작!")
                client_model = torch.load('/model/global_model.pt')

                client_dataset = torch.load('/data/cifar10_split_1.pt')
                client_loader = DataLoader(client_dataset, batch_size=32, shuffle=True)
                criterion = nn.CrossEntropyLoss()
                client_model_1 = avg_train_client(1, client_loader, client_model, 5, 0.01)

                file_path = '/model/client_model_1.pt'
                torch.save(client_model_1, file_path)
                print("클라이언트 학습 완료. 클라이언트 모델 저장")
                s.sendall(str(1).encode())
            else:
                s.sendall(str(0).encode())
        except socket.error as e:
            print(f"학습 오류 발생: {e}")
        
except socket.error as e:
    print(f"서버와의 연결에 실패했습니다: {e}")

finally:
    if s:
        s.close()
        print("소켓 닫음")

서버와 연결되었습니다.
test1 b'0'
정보를 서버에게 보냈습니다.
소켓 닫음


KeyboardInterrupt: 

In [19]:
criterion = nn.CrossEntropyLoss()

In [48]:
def log_message(message):
    current_time = time.strftime("%M:%S", time.gmtime())  # [mm:ss] 포맷으로 시간 가져오기
    print(f"[{current_time}] {message}")

In [ ]:
import socket
import time
import multiprocessing

server_address = ('10.3.129.180', 9090)
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(server_address)
id = 0



def getId():
    data = s.recv(1024).decode('utf-8')
    id = int(data)
    print(f"client id : {id}")
    return id

def recvFile(fileName):
    with open(fileName, 'wb') as f:
        print(log_message(f"Receiving from server {fileName}"))
        while True:
            data = s.recv(4096)
            if not data:
                break
            f.write(data)
    log_message(f"{fileName} received and saved")

def sendFile(fileName):
    with open(fileName, 'rb') as f:
        log_message(f"Sending {fileName} to server")
        while True:
            data = f.read(4096)
            if not data:
                break
            s.sendall(data)
    log_message(f"{fileName} sent to server")

def Learning(id, round):


    model = resnet18(10, False)
    model = torch.load('/mnt/parameters/global_model.pt') # 이 부분 나중에 수정
    model.to(device)


    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

    for epoch in range(1):
        log_message('Round {}  Client {} Training Success'.format(round, id))
        for (i, (x,y)) in enumerate(train_loader_CIFAR10):
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            local_out, _ = model(x)
      
            loss = criterion(local_out, y)
            loss.backward()
            optimizer.step()

    torch.save(model, f'output/client_model_{id}.pt')
    
def getUpdatedPT(id, round): # 여기서 end 메시지도 받음
    recvFile(f"client_{id}_global_model.pt")

    if(msg=="end"):
        return "end"
    
    Learning(id, round)
    sendFile(f"output/client_model_{id}.pt")
    msg = "done learning\n"
    s.sendall(msg.encode('utf-8'))


def run():
    log_message("시작")
    print(f"서버에 접속 중: {s.getpeername()}")
    id = getId()
    round = 1
    while True:
        
        t = getUpdatedPT(id, round)
        round += 1
        if(t=="end"):
            log_message("종료 코드 수신")
            s.sendall(msg.encode('utf-8'))
            break


if __name__ == "__main__":
    run()

[36:09] 시작
서버에 접속 중: ('10.3.129.180', 9090)
client id : 1


/tmp/ipykernel_81/1312152969.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/mnt/parameters/global_model.pt')


[36:09] Round 1  Client 1 Training Success
[36:16] Round 2  Client 1 Training Success
[36:23] Round 3  Client 1 Training Success
[36:30] Round 4  Client 1 Training Success
[36:37] Round 5  Client 1 Training Success
[36:44] Round 6  Client 1 Training Success
[36:51] Round 7  Client 1 Training Success
[36:59] Round 8  Client 1 Training Success
[37:05] Round 9  Client 1 Training Success
[37:12] Round 10  Client 1 Training Success
[37:19] Round 11  Client 1 Training Success
[37:26] Round 12  Client 1 Training Success
